In [1]:
import pandas as pd
import numpy as np

%matplotlib tk
# %matplotlib inline
import matplotlib.pyplot as plt

Import and inspect data

In [14]:
# read in data
data_folder = "D:/OneDrive/Code/WaveGVS/Data/005/"
data_file = data_folder + "005_stepGVS_zero-one-two-headtilted_20180814_094807.txt"
df = pd.read_csv(data_file, sep="; ", engine="python")

In [15]:
df.head(10)

,trial_nr,current,line_offset,line_ori
0,1,2.0,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2,0.0,-2,"[-1.25, -1.25, -1.25, -1.25, -1.25, -1.25, -1...."
2,3,2.0,-2,"[-5.5, -5.5, -5.5, -5.5, -5.5, -5.5, -5.5, -5...."
3,4,2.0,3,"[-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2...."
4,5,1.0,2,"[-1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1...."
5,6,2.0,1,"[-5.25, -5.25, -5.25, -5.25, -5.25, -5.25, -5...."
6,7,1.0,-3,"[-2.5, -2.5, -2.5, -2.5, -2.5, -2.5, -2.5, -2...."
7,8,1.0,-4,"[-5.5, -5.5, -5.5, -5.5, -5.5, -5.5, -5.5, -5...."
8,9,2.0,-3,"[-4.5, -4.5, -4.5, -4.5, -4.5, -4.5, -4.5, -4...."
9,10,1.0,-5,"[-2.5, -2.5, -2.5, -2.5, -2.5, -2.5, -2.5, -2...."


In [16]:
df.dtypes

trial_nr         int64
current        float64
line_offset      int64
line_ori        object
dtype: object

Data correction and preprocessing

In [17]:
# outer list comprehension turns string into list
# nested: turn strings inside the list into float
df["line_ori"] = [[float(y) for y in x.strip("[]").split(",")] for x in df["line_ori"]]

In [18]:
# get conditions
currents = sorted(df["current"].unique())
c = len(currents)

In [19]:
# get endpoint of each trial
df["SVV"] = np.nan
for index, row in df.iterrows():
    df.loc[index, "SVV"] = row["line_ori"][-1]

Plot data

In [24]:
# plot raw line orientation traces
plt.figure()
count = 0
for curr in currents:
    count += 1
    plt.subplot(c, 1, count)
    selection = df[(df["current"] == curr)]
    for index, row in selection.iterrows():
        orientation = row["line_ori"]
        # collapse over offset angles
        # orientation = [x - row["line_offset"] for x in orientation]
        samples = np.arange(0, len(orientation))

        plt.plot(samples, orientation)

    plt.title("{0} mA".format(curr))
    plt.ylabel("SVV (deg)")
    plt.xlabel("t (samples)")
    plt.ylim((-11, 2))
plt.tight_layout()

In [25]:
# plot endpoints per current
count = 0
x_tick_positions = []
plotlabels = []
fig, ax = plt.subplots()
jitter_width = 0.4

for curr in currents:
    count += 1
    x_tick_positions.append(count)
    plotlabels.append("{0} mA".format(curr))
    selection = df[(df["current"] == curr)]
    for index, row in selection.iterrows():
        x = count + np.random.rand() * jitter_width - (jitter_width / 2)
        plt.scatter(x, row["SVV"], color="blue", alpha=0.2)

    mean_amp = np.mean(selection["SVV"])
    std_amp = np.std(selection["SVV"])
    plt.errorbar(count, mean_amp, yerr=std_amp, ecolor="red", alpha=0.5)
    plt.scatter(count, mean_amp, color="red", alpha=0.5)

    plt.xticks(x_tick_positions)
    ax.set_xticklabels(plotlabels)
    plt.ylabel("final SVV (deg)")
    plt.title("line setting at end of trial")
plt.tight_layout()